In [253]:
import pandas as pd
import numpy as np
import random
import string

def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    rand_int = random.randint(0,2)
#     print(rand_int)
    string_random = ''.join(random.choice(letters) for i in range(stringLength))
#     for i in range(rand_int+1):
    
#         string_random = string_random +" " + ''.join(random.choice(letters) for i in range(stringLength))
    
    return "Destination_"+string_random

print(randomString())

Destination_fpfshfijtv


In [254]:
print(randomString())

Destination_carzefrffm


In [255]:
data = open("dummyData.txt").readlines()

In [256]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [257]:
array = []
for textindex,text in enumerate(data):
    doc = nlp(text.strip())
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #        token.shape_, token.is_alpha, token.is_stop)
        text = token.text
        if token.text == "XYYZ":
            tag = "Destination"
            text = randomString()
        else:
            tag = ""
        
        array.append(dict(Sentence=textindex,Word=text,POS=token.pos_,Tag=tag))

In [258]:
df = pd.DataFrame(array)  

In [259]:
df.tail(10)

,POS,Sentence,Tag,Word
770,AUX,121,,get
771,NOUN,121,,directions
772,ADV,121,,home
773,PRON,122,,I
774,VERB,122,,got
775,PART,122,,to
776,AUX,122,,get
777,NOUN,122,,directions
778,ADP,122,,to
779,PROPN,122,Destination,Destination_lekhjrjydy


In [260]:
words = list(set(df["Word"].values))
words.append("ENDPAD")
words.append("Unknown")
n_words = len(words); n_words

133

In [261]:
tags = list(set(df["Tag"].values))
n_tags = len(tags); n_tags

pos = list(set(df["POS"].values))
pos_tags = len(pos)

In [262]:
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                   s["POS"].values.tolist(),
                                                   s["Tag"].values.tolist())]
grouped = df.groupby("Sentence").apply(agg_func)

In [263]:
sentences = [s for s in grouped]

In [264]:
print(sentences[0])

[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Destination_lvjvwfmxhg', 'PROPN', 'Destination')]


In [265]:
def yield_sentence():
    for group in grouped:
        yield group
    yield None

In [266]:
for group in yield_sentence():
    print(group)
    print("-------")

[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Destination_lvjvwfmxhg', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Destination_icdhfauyhd', 'PROPN', 'Destination'), ('home', 'ADV', '')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Destination_kqmboocojq', 'PROPN', 'Destination'), ('work', 'NOUN', '')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('the', 'DET', ''), ('closest', 'ADJ', ''), ('Destination_yunpiumlsg', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('the', 'DET', ''), ('nearest', 'ADJ', ''), ('Destination_ubmxibjijg', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('home', 'ADV', '')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('to', 'ADP', ''), ('Destination_twftmbmnuo', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('to', 'ADP', 

In [267]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words) if "Destination_" not in w}
tag2idx = {t: i for i, t in enumerate(tags)}


In [268]:
word2idx["directions"]

121

In [269]:
tag2idx[""]

0

In [270]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx.get(w[0],word2idx["Unknown"]) for w in s] for s in sentences]

In [271]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)


In [272]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [273]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx[""])


In [274]:
from keras.utils import to_categorical

In [275]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [276]:
from sklearn.model_selection import train_test_split

In [277]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [278]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import keras

In [279]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [280]:
model = Model(input, out)

In [281]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

/home/premraj/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/premraj/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [282]:
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 75)                0         
_________________________________________________________________
embedding_16 (Embedding)     (None, 75, 20)            2680      
_________________________________________________________________
bidirectional_9 (Bidirection (None, 75, 100)           28400     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 75, 50)            5050      
_________________________________________________________________
crf_9 (CRF)                  (None, 75, 2)             110       
Total params: 36,240
Trainable params: 36,240
Non-trainable params: 0
_________________________________________________________________


In [283]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=60,
                    validation_split=0.1, verbose=1)

/home/premraj/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 99 samples, validate on 11 samples
Epoch 1/60
99/99 [==============================] - 2s 24ms/step - loss: 1.5149 - crf_viterbi_accuracy: 0.0100 - val_loss: 0.8754 - val_crf_viterbi_accuracy: 0.0109
Epoch 2/60
99/99 [==============================] - 0s 4ms/step - loss: 0.4114 - crf_viterbi_accuracy: 0.6895 - val_loss: 0.0396 - val_crf_viterbi_accuracy: 0.9891
Epoch 3/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0387 - crf_viterbi_accuracy: 0.9890 - val_loss: 0.0372 - val_crf_viterbi_accuracy: 0.9891
Epoch 4/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0361 - crf_viterbi_accuracy: 0.9900 - val_loss: 0.0365 - val_crf_viterbi_accuracy: 0.9891
Epoch 5/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0339 - crf_viterbi_accuracy: 0.9900 - val_loss: 0.0340 - val_crf_viterbi_accuracy: 0.9891
Epoch 6/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0329 - crf_viterbi_accuracy: 0.9892 - val_loss: 0.0328 - val_

99/99 [==============================] - 0s 3ms/step - loss: -0.0122 - crf_viterbi_accuracy: 1.0000 - val_loss: -0.0121 - val_crf_viterbi_accuracy: 1.0000
Epoch 51/60
99/99 [==============================] - 0s 3ms/step - loss: -0.0122 - crf_viterbi_accuracy: 1.0000 - val_loss: -0.0124 - val_crf_viterbi_accuracy: 1.0000
Epoch 52/60
99/99 [==============================] - 0s 4ms/step - loss: -0.0124 - crf_viterbi_accuracy: 1.0000 - val_loss: -0.0124 - val_crf_viterbi_accuracy: 1.0000
Epoch 53/60
99/99 [==============================] - 0s 4ms/step - loss: -0.0125 - crf_viterbi_accuracy: 1.0000 - val_loss: -0.0125 - val_crf_viterbi_accuracy: 1.0000
Epoch 54/60
99/99 [==============================] - 0s 3ms/step - loss: -0.0125 - crf_viterbi_accuracy: 1.0000 - val_loss: -0.0126 - val_crf_viterbi_accuracy: 1.0000
Epoch 55/60
99/99 [==============================] - 0s 4ms/step - loss: -0.0126 - crf_viterbi_accuracy: 1.0000 - val_loss: -0.0126 - val_crf_viterbi_accuracy: 1.0000
Epoch 56/6

In [284]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [285]:
test_pred = model.predict(X_te, verbose=1)

13/13 [==============================] - 0s 22ms/step


In [248]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", ""))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [250]:
print(test_labels)
print("-----")
print(pred_labels)
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

[['', '', '', '', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', 'Destination', '', '', '', '', '', '', '', 

IndexError: string index out of range

In [291]:
sentence = "Take me quickly to police station"
entities_words = ["police","station"]
array = []
doc = nlp(sentence)
for token in doc:
    text = token.text
    print(text)
    if token.text in entities_words:
        tag = "Destination"
    else:
        tag = ""

    array.append(tuple([token.text,token.pos_,tag]))
print(array)
X = [[word2idx.get(w[0],word2idx["Unknown"]) for w in s] for s in [array]]
print(X)
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
y = [[tag2idx[w[2]] for w in s] for s in [array]]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx[""])
y = [to_categorical(i, num_classes=n_tags) for i in y]
#print(y)
test_pred = model.predict(X, verbose=1)
pred_labels = pred2label(test_pred)
test_labels = pred2label(y)
print(test_labels,pred_labels)
#print(pred_labels)

Take
me
quickly
to
police
station
[('Take', 'VERB', ''), ('me', 'PRON', ''), ('quickly', 'ADV', ''), ('to', 'ADP', ''), ('police', 'NOUN', 'Destination'), ('station', 'NOUN', 'Destination')]
[[133, 108, 133, 99, 133, 133]]
1/1 [==============================] - 0s 22ms/step
[['', '', '', '', 'Destination', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']] [['Destination', '', 'Destination', '', 'Destination', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]


In [290]:
print(word2idx)

{'nearest': 8, 'have': 10, 'go': 11, 'want': 12, 'closest': 15, "'s": 23, 'kids': 26, 'parents': 28, 'kid': 31, 'let': 32, 'I': 39, 'get': 42, 'work': 53, "'": 54, 'my': 62, 'a': 64, 'one': 65, 'drive': 72, 'need': 76, 'children': 78, ',': 81, 'the': 82, 'school': 88, 'to': 99, 'place': 103, 'me': 108, 'home': 111, 'XYYZlet': 115, 'navigate': 119, 'got': 120, 'directions': 121, 'office': 127, 'ENDPAD': 132, 'Unknown': 133}
